In [1]:
import pandas as pd

# helps convert zip to lat and long
from pyzipcode import ZipCodeDatabase

In [293]:
# needed columns for discipline,dropout, gradrate, and readygrad
discipline_fields =['students_total','district_name','percent_suspended',
                    'percent_expelled','percent_committed_zero_tolerance','school_name']
dropout_fields =['district_name','school_name','dropout_rate','subgroup']
grad_fields =['district_name','graduation rate ','school_name','subgroup']
gradready_fields =['district_name','school_name','subgroup',
                   'percentage of ready graduates','number of students total']
act_fields =['district_name','school_name','participation_rate','average_english_score','subgroup',
             'average_math_score','average_reading_score','average_science_score',
             'average_composite_score','percent_scoring_21_or_higher','percent_scoring_below_19']
school_fields =['school_name','zipcode']

In [294]:

# 2020 report data on discipline
# usecols=discipline_fields
discipline_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','overall_discipline',
                              usecols=discipline_fields
                              )

# school data

school_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','school_info',
                              usecols=school_fields
                              )


# dropout_district
# usecols=dropout_fields
dropout_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','dropout_school',
                           usecols=dropout_fields
                          )
# grad_rate_district
# usecols=grad_fields
gradrate_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','grad_rate_school'
                           ,usecols=grad_fields)
# ready_grad_district
# usecols=gradready_fields
readygrad_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','ready_grad_school'
                          ,usecols=gradready_fields )
act_df = pd.read_excel('../data/original_data/reportcarddata2020.xlsx','ACT_school',
                      usecols=act_fields)

# needed
## report data needs to filter to davidson county and merge by state
### drop rows that are not needed

# clean shchool data

In [295]:
zipcode_database = ZipCodeDatabase()

In [296]:
def get_long(zip):
    try:
        return zipcode_database[zip].longitude
    except:
        pass
def get_lat(zip):
    try:
        return zipcode_database[zip].latitude
    except:
        pass
school_df['longitude'] = school_df['zipcode'].apply(get_long)
school_df['latitude'] = school_df['zipcode'].apply(get_lat)

In [297]:
school_df.shape

(1878, 4)

# clean discipline data

In [299]:
discipline_df = discipline_df.iloc[1: , :]

In [300]:
discipline_df=discipline_df.dropna()

In [301]:
# counting the duplicates
#dups = discipline_df.pivot_table(index = ['school_name'], aggfunc ='size')

In [302]:
# dropping duplicate values
#discipline_df=discipline_df.drop_duplicates(subset=['school_name'])

In [303]:
discipline_df.shape

(1797, 6)

# merge discipline and school data

In [305]:
# merge by school_name
merge_a=discipline_df.merge(school_df,how='right', on="school_name")

In [306]:
merge_a.shape

(2074, 9)

In [307]:
# dropping duplicate values
merge_a=merge_a.drop_duplicates(subset=['school_name'])

In [308]:
merge_a.shape

(1812, 9)

In [309]:
merge_a=merge_a.dropna()

In [310]:
merge_a.shape

(1731, 9)

# clean dropout data

In [313]:
dropout_df.isnull().sum()

district_name     9
school_name       9
subgroup          0
dropout_rate     47
dtype: int64

In [312]:
dropout_df=dropout_df[dropout_df.subgroup =='All Students']

In [319]:
dropout_df.shape

(372, 2)

In [315]:
dropout_df=dropout_df.dropna()

In [320]:
dropout_df.head(5)

,school_name,dropout_rate
0,Anderson County High School,1.4
14,Clinton High School,3.7
38,Oak Ridge High School,6.6
55,Cascade High School,0.9
71,Shelbyville Central High School,8.7


In [318]:
dropout_df=dropout_df.drop(['district_name', 'subgroup'], axis=1)

# clean graduate data

In [321]:
gradrate_df=gradrate_df[gradrate_df.subgroup =='All Students']

In [322]:
# remove *
gradrate_df['graduation rate ']= gradrate_df['graduation rate '].replace('*','0')

<ipython-input-322-e223408d1fa8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gradrate_df['graduation rate ']= gradrate_df['graduation rate '].replace('*','0')


In [323]:
# remove **
gradrate_df['graduation rate ']= gradrate_df['graduation rate '].replace('**','0')

<ipython-input-323-bcac58e43fa5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gradrate_df['graduation rate ']= gradrate_df['graduation rate '].replace('**','0')


In [324]:
gradrate_df['graduation rate ']=gradrate_df['graduation rate '].astype(float)

<ipython-input-324-6bf4b6800f7b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gradrate_df['graduation rate ']=gradrate_df['graduation rate '].astype(float)


In [325]:
gradrate_df.head(5)

,district_name,school_name,subgroup,graduation rate
0,Anderson County,Anderson County High School,All Students,96.1
17,Anderson County,Clinton High School,All Students,95.2
36,Anderson County,Clinch River Community School,All Students,0.0
46,Oak Ridge,Oak Ridge High School,All Students,92.1
66,Bedford County,Cascade High School,All Students,98.2


In [326]:
#gradrate_df=gradrate_df.drop(['district_name','subgroup'], axis=1)

# merge grad rate and dropout

In [327]:
# merge drop out and grad rate
merge_b=gradrate_df.merge(dropout_df,how='right', on="school_name")

In [331]:
merge_b.shape

(364, 5)

In [330]:
# dropping duplicate values
merge_b=merge_b.drop_duplicates(subset=['school_name'])

In [240]:
# drop columns
#merge_b=merge_b.drop(['district_name_y', 'subgroup_y'], axis=1)

In [332]:
# counting the duplicates
#dups = merge_b.pivot_table(index = ['school_name'], aggfunc ='size')

# merge a and b

In [333]:
merge_a.head(5)

,district_name,school_name,students_total,percent_suspended,percent_expelled,percent_committed_zero_tolerance,zipcode,longitude,latitude
0,Anderson County,Anderson County High School,1144.0,2.5,0.0,0.5,37716,-84.14656,36.099283
2,Anderson County,Andersonville Elementary,328.0,0.0,0.0,0.0,37705,-84.01131,36.217896
3,Anderson County,Briceville Elementary,117.0,1.7,0.0,0.0,37710,-84.28968,36.152310
4,Anderson County,Claxton Elementary,465.0,1.5,0.0,0.0,37849,-84.04192,36.044507
5,Anderson County,Clinton Middle School,687.0,11.1,0.0,0.1,37716,-84.14656,36.099283


In [336]:
merge_b.head(5)

,school_name,graduation rate,dropout_rate
0,Anderson County High School,96.1,1.4
1,Clinton High School,95.2,3.7
2,Oak Ridge High School,92.1,6.6
3,Cascade High School,98.2,0.9
4,Shelbyville Central High School,88.7,8.7


In [335]:
merge_b=merge_b.drop(['district_name', 'subgroup'], axis=1)

In [337]:
# merge 
merge_c=merge_a.merge(merge_b,how='right', on="school_name")

In [339]:
merge_c.shape

(364, 11)

In [341]:
merge_c.head(5)

,district_name,school_name,students_total,percent_suspended,percent_expelled,percent_committed_zero_tolerance,zipcode,longitude,latitude,graduation rate,dropout_rate
359,Achievement School District,Pathways in Education - TN,192.0,0.0,0.0,0.0,38127.0,-90.00646,35.223796,11.8,52.9
360,Achievement School District,Pathways in Education - Whitehaven,244.0,0.0,0.0,0.0,38116.0,-90.01128,35.033190,11.0,36.7
361,Achievement School District,Fairley High School,497.0,18.3,0.2,1.0,38109.0,-90.07523,35.050286,80.5,15.2
362,Achievement School District,Martin Luther King Preparatory High School,681.0,30.2,0.0,1.5,38127.0,-90.00646,35.223796,73.0,22.3
363,Achievement School District,Hillcrest High School,559.0,6.4,0.0,0.9,38116.0,-90.01128,35.033190,60.4,30.9


# clean readygrad_df

In [342]:
readygrad_df=readygrad_df[readygrad_df.subgroup =='All Students']

In [343]:
readygrad_df['percentage of ready graduates']= readygrad_df['percentage of ready graduates'].replace('*','0')

<ipython-input-343-c32b4b8656d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readygrad_df['percentage of ready graduates']= readygrad_df['percentage of ready graduates'].replace('*','0')


In [344]:
readygrad_df['percentage of ready graduates']= readygrad_df['percentage of ready graduates'].replace('**','0')

<ipython-input-344-bed3b0b863ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readygrad_df['percentage of ready graduates']= readygrad_df['percentage of ready graduates'].replace('**','0')


In [345]:
readygrad_df['percentage of ready graduates']=readygrad_df['percentage of ready graduates'].astype(float)

<ipython-input-345-e94b3c9c8ee0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  readygrad_df['percentage of ready graduates']=readygrad_df['percentage of ready graduates'].astype(float)


In [346]:
readygrad_df.shape

(403, 5)

In [348]:
readygrad_df.tail(3)

,district_name,school_name,subgroup,number of students total,percentage of ready graduates
4853,Achievement School District,Fairley High School,All Students,164,0.0
4865,Achievement School District,Martin Luther King Preparatory High School,All Students,148,0.0
4877,Achievement School District,Hillcrest High School,All Students,149,0.0


In [370]:
readygrad_df=readygrad_df.drop(['district_name', 'subgroup'], axis=1)

# ACT CLEAN

In [349]:
act_df=act_df[act_df.subgroup =='All Students']

In [350]:
act_df=act_df.drop([ 'subgroup'], axis=1)

In [351]:
act_df=act_df.dropna()

## replace *

In [352]:
act_df['participation_rate']= act_df['participation_rate'].replace('*','0')

In [353]:
act_df['average_english_score']= act_df['average_english_score'].replace('*','0')

In [354]:
act_df['average_math_score']= act_df['average_math_score'].replace('*','0')

In [355]:
act_df['average_reading_score']= act_df['average_reading_score'].replace('*','0')

In [356]:
act_df['average_science_score']= act_df['average_science_score'].replace('*','0')

In [357]:
act_df['average_composite_score']= act_df['average_composite_score'].replace('*','0')

In [358]:
act_df['percent_scoring_21_or_higher']= act_df['percent_scoring_21_or_higher'].replace('*','0')

In [359]:
act_df['percent_scoring_below_19']= act_df['percent_scoring_below_19'].replace('*','0')

## replace **

In [360]:
act_df['participation_rate']= act_df['participation_rate'].replace('**','0')

In [361]:
act_df['average_english_score']= act_df['average_english_score'].replace('**','0')

In [362]:
act_df['average_math_score']= act_df['average_math_score'].replace('**','0')

In [363]:
act_df['average_reading_score']= act_df['average_reading_score'].replace('**','0')

In [364]:
act_df['average_science_score']= act_df['average_science_score'].replace('**','0')

In [365]:
act_df['average_composite_score']= act_df['average_composite_score'].replace('**','0')

In [366]:
act_df['percent_scoring_21_or_higher']= act_df['percent_scoring_21_or_higher'].replace('**','0')

In [367]:
act_df['percent_scoring_below_19']= act_df['percent_scoring_below_19'].replace('**','0')

In [372]:
act_df.head(5)

,district_name,school_name,participation_rate,average_english_score,average_math_score,average_reading_score,average_science_score,average_composite_score,percent_scoring_21_or_higher,percent_scoring_below_19
0,Anderson County,Anderson County High School,100,19.8,19.4,20.7,20.1,20.1,42.8,42.8
4,Anderson County,Clinton High School,100,18.3,18,19.1,19.1,18.8,32.2,53.9
9,Oak Ridge,Oak Ridge High School,97,23,22.2,24.2,23.2,23.3,64.3,23.9
14,Bedford County,Cascade High School,99,20.7,19.5,21.3,21.1,20.8,46.8,34.2
19,Bedford County,Shelbyville Central High School,96,17.3,18.1,18.5,18.5,18.3,27.8,62.5


In [371]:
readygrad_df.head(5)

,school_name,number of students total,percentage of ready graduates
0,Anderson County High School,280,50.0
12,Clinton High School,272,35.7
26,Oak Ridge High School,304,65.5
41,Cascade High School,114,45.6
55,Shelbyville Central High School,390,24.9


In [84]:
readygrad_df.shape

(403, 6)

In [373]:
# merge drop out and grad rate
merge_d=readygrad_df.merge(act_df,how='right', on="school_name")

In [374]:
merge_d.shape

(411, 12)

In [376]:
#merge_c.head(15)

In [380]:
merge_d.shape

(411, 12)

In [379]:
merge_c.shape

(364, 11)

In [381]:
merge_c=merge_c.drop(['district_name'], axis=1)

In [383]:
# final merge
merge_e=merge_c.merge(merge_d,how='right', on="school_name")

In [385]:
merge_e.shape

(411, 21)

In [395]:
merge_e.shape

(354, 21)

In [389]:
merge_e= merge_e.dropna()

In [393]:
merge_e=merge_e.drop_duplicates(subset=['school_name'])

In [398]:
# convert to a csv
merge_e.to_csv('clean_data.csv', index=False)